In [12]:
#library required

import os
import time 
from datetime import datetime
import io 
import sys

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np



In [13]:
#Weekly Roll up data with organic & inorganic share:


#Loading the data from s3 csv

# Define the file path
file_path = r"C:\Users\Poovesh Kumar M\Downloads\Planning\\1_Data\demand_planning_MPBH_29_08_2024.csv"

# Read the CSV file into a DataFrame
df_base = pd.read_csv(file_path)

# Display the DataFrame
print(df_base)



                 brand sales_channel  sales_date sale_month_year        asin  \
0        majestic pure    Amazon.com  2024-07-28         07-2024  B00QR7FTLU   
1        majestic pure    Amazon.com  2024-07-24         07-2024  B00QR7FTLU   
2        majestic pure    Amazon.com  2024-07-20         07-2024  B00QR7FTLU   
3        majestic pure    Amazon.com  2024-07-16         07-2024  B00QR7FTLU   
4        majestic pure    Amazon.com  2024-07-12         07-2024  B00QR7FTLU   
...                ...           ...         ...             ...         ...   
312535  botanic hearth    Amazon.com  2023-03-27         03-2023  B010CA0M50   
312536  botanic hearth    Amazon.com  2023-03-28         03-2023  B010CA0M50   
312537  botanic hearth    Amazon.com  2023-03-29         03-2023  B010CA0M50   
312538  botanic hearth    Amazon.com  2023-03-30         03-2023  B010CA0M50   
312539  botanic hearth    Amazon.com  2023-03-31         03-2023  B010CA0M50   

       earliest_date_brand_channel curr

C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_4292\506433516.py:10: DtypeWarning: Columns (14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_base = pd.read_csv(file_path)


# 2. Filter (df2): brand x channel level into individual dataframes
df_bh_us = df_base[(df_base['brand'] == 'botanic hearth') & (df_base['sales_channel'] == 'Amazon.com')]
df_bh_ca = df_base[(df_base['brand'] == 'botanic hearth') & (df_base['sales_channel'] == 'Amazon.ca')]
df_mp_us = df_base[(df_base['brand'] == 'majestic pure') & (df_base['sales_channel'] == 'Amazon.com')]
df_mp_ca = df_base[(df_base['brand'] == 'majestic pure') & (df_base['sales_channel'] == 'Amazon.ca')]

# Save the brand x sales channel DataFrame to an csv file
date_str = '29_08_24'

df_bh_us.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/1_Data/df_bh_us_' + date_str + '.csv', index=False)
df_bh_ca.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/1_Data/df_bh_ca_' + date_str + '.csv', index=False)
df_mp_us.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/1_Data/df_mp_us_' + date_str + '.csv', index=False)
df_mp_ca.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/1_Data/df_mp_ca_' + date_str + '.csv', index=False)



In [83]:
# Read the CSV file into a DataFrame
df_mp_us = pd.read_csv("C:/Users/Poovesh Kumar M/Downloads/Planning/1_Data/df_mp_us_29_08_24.csv")
df_mp_us

df_bs1 = df_mp_us

# Step 2: Create a list of distinct 'brand' and 'sales_channel' combinations
brands_channel_list = [f"{brand.replace('.', '_')}_{channel.replace('.', '_')}" 
                       for brand, channel in df_bs1[['brand', 'sales_channel']].drop_duplicates().values]

# Step 3: Convert the list to a single string, separated by commas
brands_channel_str = ', '.join(brands_channel_list)

# Display the combined string
print(brands_channel_str)


majestic pure_Amazon_com


C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_4292\132196174.py:2: DtypeWarning: Columns (14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mp_us = pd.read_csv("C:/Users/Poovesh Kumar M/Downloads/Planning/1_Data/df_mp_us_29_08_24.csv")


In [84]:
#creating New DF 
df_bs2 = df_bs1
# Display the first few rows of the filtered DataFrame
print(df_bs2.head())

#creating ASINx VB for mapping purpose
# Remove rows where 'asin' is null
df_asin_vb = df_bs2.dropna(subset=['asin'])

# Select distinct 'asin' and 'velocity_bucket' combinations
df_asin_vb = df_asin_vb[['asin', 'velocity_bucket']].drop_duplicates()


# Define the replacement dictionary
replacement_dict = {
    'SF': '1_SF',
    'F': '2_F',
    'M': '3_M',
    'SM': '4_SM',
    'NULL': 'NULL'
}

df_asin_vb['velocity_bucket'] = df_asin_vb['velocity_bucket'].replace(replacement_dict)

# Display the resulting DataFrame
print("Distinct asin x velocity_bucket:")
print(df_asin_vb.head())

           brand sales_channel  sales_date sale_month_year        asin  \
0  majestic pure    Amazon.com  2024-07-28         07-2024  B00QR7FTLU   
1  majestic pure    Amazon.com  2024-07-24         07-2024  B00QR7FTLU   
2  majestic pure    Amazon.com  2024-07-20         07-2024  B00QR7FTLU   
3  majestic pure    Amazon.com  2024-07-16         07-2024  B00QR7FTLU   
4  majestic pure    Amazon.com  2024-07-12         07-2024  B00QR7FTLU   

  earliest_date_brand_channel currency        asp     gmv  inorganic_gmv  ...  \
0                  2022-10-01      USD  17.797143  249.16          17.99  ...   
1                  2022-10-01      USD  17.990000  197.89          71.96  ...   
2                  2022-10-01      USD  17.990000  125.93          35.98  ...   
3                  2022-10-01      USD  17.990000  395.78         230.77  ...   
4                  2022-10-01      USD  17.996316  341.93         125.39  ...   

       category  sub_category  product_type   theme  dros  cummulati

In [85]:
# df_weekly.columns
# df_weekly_filtered
# df_weekly

df_bs2

,brand,sales_channel,sales_date,sale_month_year,asin,earliest_date_brand_channel,currency,asp,gmv,inorganic_gmv,...,category,sub_category,product_type,theme,dros,cummulative_ros,velocity_bucket,vb_created_datetime,inorganic_gmv_units,organic_gmv_units
0,majestic pure,Amazon.com,2024-07-28,07-2024,B00QR7FTLU,2022-10-01,USD,17.797143,249.16,17.99,...,Carrier Oils,NaN,B00QR7FTLU,Legacy,16.0,0.9236,SM,2024-08-28 21:09:06,1.0108,12.9891
1,majestic pure,Amazon.com,2024-07-24,07-2024,B00QR7FTLU,2022-10-01,USD,17.990000,197.89,71.96,...,Carrier Oils,NaN,B00QR7FTLU,Legacy,16.0,0.9236,SM,2024-08-28 21:09:06,4.0000,7.0000
2,majestic pure,Amazon.com,2024-07-20,07-2024,B00QR7FTLU,2022-10-01,USD,17.990000,125.93,35.98,...,Carrier Oils,NaN,B00QR7FTLU,Legacy,16.0,0.9236,SM,2024-08-28 21:09:06,2.0000,5.0000
3,majestic pure,Amazon.com,2024-07-16,07-2024,B00QR7FTLU,2022-10-01,USD,17.990000,395.78,230.77,...,Carrier Oils,NaN,B00QR7FTLU,Legacy,16.0,0.9236,SM,2024-08-28 21:09:06,12.8276,9.1723
4,majestic pure,Amazon.com,2024-07-12,07-2024,B00QR7FTLU,2022-10-01,USD,17.996316,341.93,125.39,...,Carrier Oils,NaN,B00QR7FTLU,Legacy,16.0,0.9236,SM,2024-08-28 21:09:06,6.9675,12.0257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196972,majestic pure,Amazon.com,2024-07-14,07-2024,B09RYRHYHH,2022-10-01,USD,0.000000,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000
196973,majestic pure,Amazon.com,2024-07-18,07-2024,B09RYRHYHH,2022-10-01,USD,0.000000,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000
196974,majestic pure,Amazon.com,2024-07-22,07-2024,B09RYRHYHH,2022-10-01,USD,0.000000,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000
196975,majestic pure,Amazon.com,2024-07-26,07-2024,B09RYRHYHH,2022-10-01,USD,0.000000,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000


In [96]:
#Weekly Roll up of data:

# Define the replacement dictionary
replacement_dict = {
    'SF': '1_SF',
    'F': '2_F',
    'M': '3_M',
    'SM': '4_SM',
    'NULL': 'NULL'
}

df_bs2['velocity_bucket'] = df_bs2['velocity_bucket'].replace(replacement_dict)

# Convert 'sales_date' to datetime format if it isn't already
df_bs2['sales_date'] = pd.to_datetime(df_bs2['sales_date'])

# Extract the week and year, setting the week to start on Monday and end on Sunday
df_bs2['Week_Start_date'] = df_bs2['sales_date'].dt.to_period('W-MON').apply(lambda r: r.start_time.date())
df_bs2['Week_End_date'] = df_bs2['sales_date'].dt.to_period('W-MON').apply(lambda r: r.end_time.date())

# Select only the relevant columns
df_bs2 = df_bs2[['velocity_bucket','asin','sales_date','Week_Start_date','Week_End_date','currency', 'gmv_units','organic_gmv_units','inorganic_gmv_units',
                  'ad_spend','inventory_available', 'gmv', 'inorganic_gmv', 'organic_gmv']]

# Create a new column to flag days with inventory available
df_bs2['inventory_flag'] = df_bs2['inventory_available'] > 0

# Group by 'asin' and 'Week_Start_date' to summarize the necessary columns
df_weekly = df_bs2.groupby(['asin', 'Week_Start_date'], as_index=False).agg({
    'velocity_bucket': 'first',  # Assuming velocity_bucket doesn't change within a week for an ASIN
    'gmv_units': 'sum',
    'organic_gmv_units': 'sum',
    'inorganic_gmv_units': 'sum',
    'ad_spend': 'sum',
    'inventory_available': 'sum',
    'gmv': 'sum',
    'inorganic_gmv': 'sum',
    'organic_gmv': 'sum',
    'inventory_flag': 'sum'  # Sum the True values (which represent days with inventory)
})

# Recalculate Week_End_date since it was lost during grouping
df_weekly['Week_End_date'] = df_weekly['Week_Start_date'] + pd.to_timedelta(6, unit='d')

# Calculate the total number of days inventory data is present in that week
df_weekly = df_weekly.assign(total_days=lambda x: df_bs2.groupby(['asin', 'Week_Start_date'])['sales_date'].transform('nunique'))

# Calculate inventory_Availability% = (count of days with inventory_available > 0) / total days inventory is present * 100
df_weekly['inventory_Availability%'] = (df_weekly['inventory_flag'] / df_weekly['total_days']) * 100

# Calculate asp_week = sum(gmv) / sum(gmv_units)
df_weekly['asp_week'] = df_weekly['gmv'] / df_weekly['gmv_units']

# Select relevant columns for the final DataFrame
df_weekly = df_weekly[['velocity_bucket', 'asin', 'Week_Start_date', 'Week_End_date',
                             'organic_gmv_units', 'inorganic_gmv_units', 'asp_week', 'ad_spend','inventory_Availability%',
                               'gmv', 'inorganic_gmv', 'organic_gmv','gmv_units','inventory_available' ]]
                             
# Save the weekly report to a CSV file
# df_weekly.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/'+brands_channel_str+'_weekly_data.csv', index=False)


# df_weekly_report = df_weekly[['velocity_bucket', 'asin', 'Week_Start_date', 'Week_End_date',
#                              'organic_gmv_units', 'inorganic_gmv_units', 'asp_week', 'ad_spend','inventory_Availability%',
#                                'gmv', 'inorganic_gmv', 'organic_gmv','gmv_units','inventory_available' ]]



C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_4292\3150691763.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bs2['velocity_bucket'] = df_bs2['velocity_bucket'].replace(replacement_dict)
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_4292\3150691763.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bs2['sales_date'] = pd.to_datetime(df_bs2['sales_date'])
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_4292\3150691763.py:18: SettingWithCopyWarning: 
A value is trying to be 

In [58]:
# Filtering data for 2024
# Convert Week_Start_date to datetime if it's not already

df_weekly['Week_Start_date'] = pd.to_datetime(df_weekly['Week_Start_date'])

# Define the date range
start_date = pd.to_datetime('2024-01-01')
end_date = pd.to_datetime('2024-07-30')

# Apply the filter
df_weekly_base_data_filtered = df_weekly[
    (df_weekly['Week_Start_date'] >= start_date) & 
    (df_weekly['Week_Start_date'] < end_date)
]

df_weekly_base_data_filtered
# Save the weekly report to a CSV file
# df_weekly_base_data_filtered.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/'+brands_channel_str+'_weekly_data_20204.csv', index=False)


,velocity_bucket,asin,Week_Start_date,Week_End_date,organic_gmv_units,inorganic_gmv_units,asp_week,ad_spend,inventory_Availability%,gmv,inorganic_gmv,organic_gmv,gmv_units,inventory_available
66,SF,B00PMR3QF2,2024-01-02,2024-01-08,1065.2478,653.4107,16.123950,4128.532294,100.000000,27878.31,10517.08,17194.22,1729,408636
67,SF,B00PMR3QF2,2024-01-09,2024-01-15,1091.3222,598.8277,15.413422,3611.831894,100.000000,26218.23,9275.63,16775.30,1701,372225
68,SF,B00PMR3QF2,2024-01-16,2024-01-22,1862.3570,785.7508,13.460943,3114.869153,100.000000,35954.18,10497.69,25145.29,2671,328782
69,SF,B00PMR3QF2,2024-01-23,2024-01-29,1177.7482,500.8774,16.071342,2776.467330,100.000000,27192.71,8048.67,18928.79,1692,382362
70,SF,B00PMR3QF2,2024-01-30,2024-02-05,1193.1429,606.9726,16.009906,3354.393250,100.000000,28961.92,9718.89,19100.44,1809,401148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28640,SM,B0D8HVYNM8,2024-06-25,2024-07-01,0.0000,0.0000,NaN,0.000000,0.000000,0.00,0.00,0.00,0,0
28641,SM,B0D8HVYNM8,2024-07-02,2024-07-08,0.0000,0.0000,NaN,0.000000,0.000000,0.00,0.00,0.00,0,0
28642,SM,B0D8HVYNM8,2024-07-09,2024-07-15,0.0000,0.0000,NaN,0.000000,14.285714,0.00,0.00,0.00,0,162
28643,SM,B0D8HVYNM8,2024-07-16,2024-07-22,0.0000,0.0000,NaN,0.000000,100.000000,0.00,0.00,0.00,0,2700


In [88]:
# Removing Outlier (IQR)

# Select the required columns from the original DataFrame
df_bs3 = df_weekly[['Week_Start_date', 'asin', 'organic_gmv_units']]

# Define a function to remove outliers for each 'asin'
def remove_outliers(group):
    Q1 = group['organic_gmv_units'].quantile(0.25)
    Q3 = group['organic_gmv_units'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return group[(group['organic_gmv_units'] >= lower_bound) & (group['organic_gmv_units'] <= upper_bound)]

# Apply the function to remove outliers for each 'asin'
df_bs3_no_outliers = df_bs3.groupby('asin').apply(remove_outliers).reset_index(drop=True)

# Display the DataFrame after removing outliers
print("DataFrame after removing outliers:")
print(df_bs3_no_outliers)

DataFrame after removing outliers:
      Week_Start_date        asin  organic_gmv_units
0          2022-10-04  B00PMR3QF2           720.1551
1          2022-10-11  B00PMR3QF2           880.3120
2          2022-10-18  B00PMR3QF2           880.7177
3          2022-10-25  B00PMR3QF2          1038.8408
4          2022-11-01  B00PMR3QF2          1019.5653
...               ...         ...                ...
27346      2024-07-02  B0D8HVYNM8             0.0000
27347      2024-07-09  B0D8HVYNM8             0.0000
27348      2024-07-16  B0D8HVYNM8             0.0000
27349      2024-07-23  B0D8HVYNM8             0.0000
27350      2024-07-30  B0D8HVYNM8             0.0000

[27351 rows x 3 columns]


In [89]:
#Batches:

# df_weekly already exists and 'year_week' needs to be in datetime format
df_bs3_no_outliers['Week_Start_date'] = pd.to_datetime(df_weekly['Week_Start_date'])

# Manually specify the dates you're interested in removing
selected_dates = ['2024-05-06','2024-05-13','2024-05-20','2024-05-27','2024-06-03','2024-06-10','2024-06-17','2024-06-24',
                  '2024-07-01','2024-07-08','2024-07-15','2024-07-22','2024-07-29']



# Convert the dates to datetime format
selected_dates = pd.to_datetime(selected_dates)

# Filter the DataFrame to exclude the selected dates
df_batch1 = df_bs3_no_outliers[~df_bs3_no_outliers['Week_Start_date'].isin(selected_dates)]

# Display the resulting DataFrame
print("Filtered DataFrame with selected dates removed:")
print(df_batch1)

df_batch1.tail()

Filtered DataFrame with selected dates removed:
      Week_Start_date        asin  organic_gmv_units
0          2022-09-27  B00PMR3QF2           720.1551
1          2022-10-04  B00PMR3QF2           880.3120
2          2022-10-11  B00PMR3QF2           880.7177
3          2022-10-18  B00PMR3QF2          1038.8408
4          2022-10-25  B00PMR3QF2          1019.5653
...               ...         ...                ...
27346      2024-06-18  B0D8HVYNM8             0.0000
27347      2024-06-25  B0D8HVYNM8             0.0000
27348      2024-07-02  B0D8HVYNM8             0.0000
27349      2024-07-09  B0D8HVYNM8             0.0000
27350      2024-07-16  B0D8HVYNM8             0.0000

[27351 rows x 3 columns]


,Week_Start_date,asin,organic_gmv_units
27346,2024-06-18,B0D8HVYNM8,0.0
27347,2024-06-25,B0D8HVYNM8,0.0
27348,2024-07-02,B0D8HVYNM8,0.0
27349,2024-07-09,B0D8HVYNM8,0.0
27350,2024-07-16,B0D8HVYNM8,0.0


In [32]:
#Auto Arima Forecasting in Python
!pip install pmdarima

In [62]:
from pmdarima import auto_arima

# Ensure 'year_month' is in datetime format
df_batch1['Week_Start_date'] = pd.to_datetime(df_batch1['Week_Start_date'])

# List of ASINs to forecast and plot
selected_asins = df_batch1['asin'].unique()

# Initialize a dictionary to store the forecast results
forecast_results = {}

# Input start date for the forecast
start_date = pd.to_datetime('2024-05-06')  # Replace with your desired start date

# Loop through each ASIN and perform forecasting
for asin in selected_asins:
    # Filter the DataFrame for the current ASIN
    df_asin = df_batch1[df_batch1['asin'] == asin]
    
    if df_asin.empty:
        print(f"No data available for ASIN {asin}")
        continue

    if df_asin.shape[0] < 10:
        print(f"Not enough data for ASIN {asin} ({df_asin.shape[0]} data points)")
        continue

    if df_asin['organic_gmv_units'].isnull().any():
        print(f"Missing values found for ASIN {asin}")
        continue

    # Build the Auto ARIMA model
    model = auto_arima(df_asin['organic_gmv_units'], 
                       seasonal=False,
                       stepwise=False,
                       approximation=False,
                       trace=False, 
                       suppress_warnings=True)
    
    # Get the p, d, q values
    p, d, q = model.order
    
    # Forecast for the next 6 months
    forecast, conf_int = model.predict(n_periods=15, return_conf_int=True)
    
    # Create a DataFrame for the current ASIN's forecasts
    date_range = pd.date_range(start=start_date, periods=15, freq='W-MON')
    forecast_df = pd.DataFrame({
        'ASIN': asin,
        'forecast': forecast,
        'lower_conf_int': conf_int[:, 0],
        'upper_conf_int': conf_int[:, 1],
        'forecast_date': date_range,
        'p': p,
        'd': d,
        'q': q
    })
    
    # Append to the forecast results DataFrame
    forecast_results[asin] = forecast_df

# Concatenate all the individual ASIN forecast DataFrames
df_forecasts = pd.concat(forecast_results.values(), ignore_index=True)

# Save the forecast results to a CSV file
df_forecasts.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/'+brands_channel_str+'_Weekly_forecast_results1''.csv', index=False)

# Print part of the result
print(df_forecasts.head())



C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3

C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\sta

C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\sta

C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\sta

C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3

...

C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


Not enough data for ASIN B0CZL6HZ4S (6 data points)
Not enough data for ASIN B0CZS4JS9C (6 data points)
Not enough data for ASIN B0CZT1HV3L (8 data points)


C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(1,0,4)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
                           ^^^^^^^^^^^^^^
  File "C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\pmdarima\arima\arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
             ^^^^^^^^^^
  File "C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\t

Not enough data for ASIN B0D53QHNHP (6 data points)
Not enough data for ASIN B0D542WW3G (8 data points)
Not enough data for ASIN B0D7DKHFRG (9 data points)
Not enough data for ASIN B0D7HVN6J1 (6 data points)
Not enough data for ASIN B0D7HVZVK2 (8 data points)
Not enough data for ASIN B0D7HWNJ8J (8 data points)
Not enough data for ASIN B0D7HX5FJF (9 data points)
Not enough data for ASIN B0D7HZ6883 (8 data points)
Not enough data for ASIN B0D7HZR4XL (9 data points)
Not enough data for ASIN B0D811N8HV (9 data points)
Not enough data for ASIN B0D8HVYNM8 (6 data points)
         ASIN    forecast  lower_conf_int  upper_conf_int forecast_date  p  d  \
0  B00PMR3QF2  940.222308      311.316230     1569.128386    2024-05-06  1  0   
1  B00PMR3QF2  937.705093      291.858927     1583.551260    2024-05-13  1  0   
2  B00PMR3QF2  935.194618      272.928649     1597.460586    2024-05-20  1  0   
3  B00PMR3QF2  932.690863      254.487132     1610.894594    2024-05-27  1  0   
4  B00PMR3QF2  930.1938

C:\Users\Poovesh Kumar M\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [98]:
# Create a new DataFrame with distinct ASIN and their p, d, q values
df_asin_pdq = df_forecasts[['ASIN', 'p', 'd', 'q']].drop_duplicates()


# Create the 'model' column by concatenating p, d, and q values
df_asin_pdq['model'] = df_asin_pdq.apply(lambda row: f"({row['p']},{row['d']},{row['q']})", axis=1)

df_asin_pdq_1 = df_asin_vb[['asin', 'velocity_bucket']].merge(df_asin_pdq,left_on='asin', right_on='ASIN', how='right')


# Print part of the DataFrame to verify the results
print(df_asin_pdq_1.head())


# Save the forecast results to a CSV file
# df_asin_pdq_1.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/'+brands_channel_str+'_Weekly_Asin_model_results1''.csv', index=False)



         asin velocity_bucket        ASIN  p  d  q    model
0  B00PMR3QF2            1_SF  B00PMR3QF2  1  0  1  (1,0,1)
1  B00PV15BPW            1_SF  B00PV15BPW  0  1  2  (0,1,2)
2  B00Q7Z1UZ6             NaN  B00Q7Z1UZ6  0  0  0  (0,0,0)
3  B00Q96XGUU            4_SM  B00Q96XGUU  1  0  1  (1,0,1)
4  B00QR6SS6O             2_F  B00QR6SS6O  3  0  1  (3,0,1)


In [97]:
df_weekly

,velocity_bucket,asin,Week_Start_date,Week_End_date,organic_gmv_units,inorganic_gmv_units,asp_week,ad_spend,inventory_Availability%,gmv,inorganic_gmv,organic_gmv,gmv_units,inventory_available
0,1_SF,B00PMR3QF2,2022-09-27,2022-10-03,332.8853,212.9669,18.889982,2077.9056,0.000000,10351.71,4023.3207,6287.7693,548,0
1,1_SF,B00PMR3QF2,2022-10-04,2022-10-10,720.1551,466.2788,18.908032,4509.2899,0.000000,22576.19,8818.6648,13614.3252,1194,0
2,1_SF,B00PMR3QF2,2022-10-11,2022-10-17,880.3120,833.8866,15.829657,6408.8007,0.000000,27227.01,13141.9866,13992.1234,1720,0
3,1_SF,B00PMR3QF2,2022-10-18,2022-10-24,880.7177,622.1894,16.011567,4287.0209,0.000000,24209.49,9962.8650,14100.9650,1512,0
4,1_SF,B00PMR3QF2,2022-10-25,2022-10-31,1038.8408,650.0992,16.383682,4250.1084,0.000000,27901.41,10638.6032,17029.5868,1703,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28641,4_SM,B0D8HVYNM8,2024-07-02,2024-07-08,0.0000,0.0000,NaN,0.0000,0.000000,0.00,0.0000,0.0000,0,0
28642,4_SM,B0D8HVYNM8,2024-07-09,2024-07-15,0.0000,0.0000,NaN,0.0000,14.285714,0.00,0.0000,0.0000,0,162
28643,4_SM,B0D8HVYNM8,2024-07-16,2024-07-22,0.0000,0.0000,NaN,0.0000,100.000000,0.00,0.0000,0.0000,0,2700
28644,4_SM,B0D8HVYNM8,2024-07-23,2024-07-29,0.0000,0.0000,NaN,0.0000,100.000000,0.00,0.0000,0.0000,0,3433


In [99]:
#Accuracy: MAPR & WAPE

#df_monthly contains actual data and df_forecasts contains forecast data

# Convert 'year_week' 'forecast_date' in df_forecasts to datetime
df_weekly['Week_Start_date'] = pd.to_datetime(df_weekly['Week_Start_date'])
df_forecasts['forecast_date'] = pd.to_datetime(df_forecasts['forecast_date'])

# Define the forecast dates of interest
forecast_dates = ['2024-05-06','2024-05-13','2024-05-20','2024-05-27','2024-06-03','2024-06-10','2024-06-17','2024-06-24',
                  '2024-07-01','2024-07-08','2024-07-15','2024-07-22','2024-07-29']
forecast_dates = pd.to_datetime(forecast_dates)

# Filter the forecast data for the dates of interest
df_forecasts_filtered = df_forecasts[df_forecasts['forecast_date'].isin(forecast_dates)]

print(df_forecasts_filtered.head())

# Filter the actual data for the dates of interest
df_weekly_filtered = df_weekly[df_weekly['Week_Start_date'].isin(forecast_dates)]

print(df_weekly_filtered.head())

# Merge the actuals and forecasts on ASIN and date
df_merged = pd.merge(df_forecasts_filtered, df_weekly_filtered, left_on=['ASIN', 'forecast_date'],
                     right_on=['asin', 'Week_Start_date'])

print(df_merged.head())

# # Calculate the Absolute Percentage Error
# df_merged['absolute_percentage_error'] = abs((df_merged['forecast'] - df_merged['organic_gmv_units']) / df_merged['organic_gmv_units'])

# # Calculate Absolute Errors
# df_merged['absolute_error'] = abs(df_merged['forecast'] - df_merged['organic_gmv_units'])

# # Save the forecast results to a CSV file
# # df_merged.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/'+brands_channel_str+'_Weekly_forecast_errors.csv', index=False)


# # Calculate MAPE for each forecast date
# mape_per_date = df_merged.groupby('forecast_date')['absolute_percentage_error'].mean()

# # Calculate WAPE for each forecast date
# wape_per_date = df_merged.groupby('forecast_date').apply(
#     lambda x: df_merged.loc[x.index, 'absolute_error'].sum() / df_merged.loc[x.index, 'organic_gmv_units'].sum()
# )

# # Display MAPE and WAPE for each date
# print("MAPE for each forecast date:")
# print(mape_per_date)

# print("\nWAPE for each forecast date:")
# print(wape_per_date)

# # Calculate overall MAPE and WAPE across all forecast dates
# overall_mape = df_merged['absolute_percentage_error'].mean()
# overall_wape = df_merged['absolute_error'].sum() / df_merged['organic_gmv_units'].sum()

# print(f"\nOverall MAPE: {overall_mape:.2%}")
# print(f"Overall WAPE: {overall_wape:.2%}")


         ASIN    forecast  lower_conf_int  upper_conf_int forecast_date  p  d  \
0  B00PMR3QF2  940.222308      311.316230     1569.128386    2024-05-06  1  0   
1  B00PMR3QF2  937.705093      291.858927     1583.551260    2024-05-13  1  0   
2  B00PMR3QF2  935.194618      272.928649     1597.460586    2024-05-20  1  0   
3  B00PMR3QF2  932.690863      254.487132     1610.894594    2024-05-27  1  0   
4  B00PMR3QF2  930.193812      236.500690     1623.886934    2024-06-03  1  0   

   q  
0  1  
1  1  
2  1  
3  1  
4  1  
Empty DataFrame
Columns: [velocity_bucket, asin, Week_Start_date, Week_End_date, organic_gmv_units, inorganic_gmv_units, asp_week, ad_spend, inventory_Availability%, gmv, inorganic_gmv, organic_gmv, gmv_units, inventory_available]
Index: []
Empty DataFrame
Columns: [ASIN, forecast, lower_conf_int, upper_conf_int, forecast_date, p, d, q, velocity_bucket, asin, Week_Start_date, Week_End_date, organic_gmv_units, inorganic_gmv_units, asp_week, ad_spend, inventory_Availab

In [95]:
# df_weekly_filtered.tail()
# df_merged
df_weekly.tail()
# df_forecasts
# df_forecasts_filtered
# df_weekly_filtered

,velocity_bucket,asin,Week_Start_date,Week_End_date,organic_gmv_units,inorganic_gmv_units,asp_week,ad_spend,inventory_Availability%,gmv,inorganic_gmv,organic_gmv,gmv_units,inventory_available
28641,SM,B0D8HVYNM8,2024-07-02,2024-07-08,0.0,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,0,0
28642,SM,B0D8HVYNM8,2024-07-09,2024-07-15,0.0,0.0,NaN,0.0,14.285714,0.0,0.0,0.0,0,162
28643,SM,B0D8HVYNM8,2024-07-16,2024-07-22,0.0,0.0,NaN,0.0,100.000000,0.0,0.0,0.0,0,2700
28644,SM,B0D8HVYNM8,2024-07-23,2024-07-29,0.0,0.0,NaN,0.0,100.000000,0.0,0.0,0.0,0,3433
28645,SM,B0D8HVYNM8,2024-07-30,2024-08-05,0.0,0.0,NaN,0.0,28.571429,0.0,0.0,0.0,0,987


In [ ]:
#Accuracy: MAPR & WAPE at velocity_bucket


df_weekly_filtered1 = df_asin_vb[['asin', 'velocity_bucket']].merge(df_weekly_filtered, on='asin', how='right')
df_forecasts_filtered1 = df_asin_vb[['asin', 'velocity_bucket']].merge(df_forecasts_filtered,left_on='asin', right_on='ASIN', how='right')


# Define the replacement dictionary
replacement_dict = {
    'SF': '1_SF',
    'F': '2_F',
    'M': '3_M',
    'SM': '4_SM',
    'NULL': 'NULL'
}

# Replace values in the 'velocity_bucket' column
df_weekly_filtered1['velocity_bucket'] = df_weekly_filtered1['velocity_bucket'].replace(replacement_dict)
df_forecasts_filtered1['velocity_bucket'] = df_forecasts_filtered1['velocity_bucket'].replace(replacement_dict)


# Group by 'velocity_bucket' and 'year_month' to summarize 'gmv_units'
df_weekly_filtered2 = df_weekly_filtered1.groupby(['velocity_bucket', 'year_week'], as_index=False)['organic_gmv_units'].sum()
df_forecasts_filtered2 = df_forecasts_filtered1.groupby(['velocity_bucket', 'forecast_date'], as_index=False)['forecast'].sum()


# Merge the actuals and forecasts on ASIN and date
df_merged1 = pd.merge(df_forecasts_filtered2, df_weekly_filtered2, left_on=['velocity_bucket', 'forecast_date'],
                     right_on=['velocity_bucket', 'year_week'])

# Calculate the Absolute Percentage Error
df_merged1['absolute_percentage_error'] = abs((df_merged1['forecast'] - df_merged1['organic_gmv_units']) / df_merged1['organic_gmv_units'])

# Calculate Absolute Errors
df_merged1['absolute_error'] = abs(df_merged1['forecast'] - df_merged1['organic_gmv_units'])

# # Calculate MAPE for each forecast date
# mape_per_date_bucket = df_merged1.groupby(['forecast_date', 'velocity_bucket'])['absolute_percentage_error'].mean().reset_index(name='MAPE')


wape_per_date_bucket = df_merged1.groupby(['forecast_date', 'velocity_bucket']).apply(
    lambda x: x['absolute_error'].sum() / x['organic_gmv_units'].sum()
).reset_index(name='WAPE')

# # Display MAPE and WAPE for each date
# print("MAPE for each forecast date:")
# print(mape_per_date)

print("\nWAPE for each forecast date:")
print(wape_per_date_bucket)

# Calculate overall MAPE and WAPE across all forecast dates
# overall_mape = df_merged['absolute_percentage_error'].mean()
overall_wape = df_merged['absolute_error'].sum() / df_merged['organic_gmv_units'].sum()

print(f"\nOverall MAPE: {overall_mape:.2%}")
print(f"Overall WAPE: {overall_wape:.2%}")

# Save the forecast results to a CSV file
wape_per_date_bucket.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_buckets'+brands_channel_str+'.csv', index=False)
